In [ ]:
import pandas as pd

In [ ]:
%run ../ssb_sparktools/editing/editing.py

In [ ]:
%run ../ssb_sparktools/quality/quality.py

In [ ]:
%run ../ssb_sparktools/processing/processing.py

In [ ]:
skattRaw = spark.read.path('/produkt/skatt/person/temp/tverrsnitt_skattemelding')

In [ ]:
skattRaw = spark.read.path('/produkt/skatt/person/temp/tverrsnitt_skattemelding1000rader')

In [ ]:
skattRaw.count()

In [ ]:
spark.read.path('/produkt/skatt/person/temp/tverr*').show(20, False)

In [ ]:
skatt_dict = unpack_parquet(skattRaw, rootdf=True, rootvar=True)

In [ ]:
#Kjører missing_df for alle skatteobjekter og samler resultatet i en dictionary
skatt_missing = {}
for k in skatt_dict.keys():
    skatt_missing[k] = missing_df(skatt_dict[k])

In [ ]:
#Eksempel på resultat fra missing_df
skatt_missing['rootdf']

In [ ]:
#Enda et eksempel på resultat fra missing_df
skatt_missing['fondskonto']

In [ ]:
#Kjører korrigering på bolske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_bool, missing_boolcount = missing_correction_bool(skatt_dict['fondskonto'])
for k, v in missing_boolcount.items():
    print(k, v)

In [ ]:
#Eksempel før korrigering
skatt_dict['fondskonto'][['identifikator', 'overfoertFraBarn']].toPandas()

In [ ]:
#Eksempel etter korrrigering
df_corrected_bool[['identifikator', 'overfoertFraBarn']].toPandas()

In [ ]:
#Eksempel før korrigering der overfoertFraBarn skal unntas korrigering
skatt_dict['konto'].select('identifikator', 'erBoligsparekontoIAnnenEOeSStat', 'overfoertFraBarn').toPandas()

In [ ]:
#Kjører korrigering på bolske verdier på et gitt datasett med unntak for 1 variabel. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_bool, missing_boolcount = missing_correction_bool(skatt_dict['konto'], exception_for=['overfoertFraBarn'])

In [ ]:
#Eksempel før korrigering der overfoertFraBarn skal unntas korrigering
df_corrected_bool.select('identifikator', 'erBoligsparekontoIAnnenEOeSStat', 'overfoertFraBarn').toPandas()

In [ ]:
#Kjører korrigering på numeriske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_number, missing_numbcount = missing_correction_number(skatt_dict['fondskonto'])
for k, v in missing_numbcount.items():
    print(k, v)

In [ ]:
#Eksempel før korrigering
skatt_dict['fondskonto']\
                [['identifikator', 'skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel','skattepliktigTapVedRealisasjonAvOgUttakFraRentedel',\
                 'verdiFoerVerdsettingsrabattForAksjeOgAksjefond', 'skattefrittUttak', 'benyttetSkjermingsfradrag']]\
                .toPandas()

In [ ]:
#Eksempel etter korrigering
df_corrected_number\
                [['identifikator', 'skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel','skattepliktigTapVedRealisasjonAvOgUttakFraRentedel',\
                 'verdiFoerVerdsettingsrabattForAksjeOgAksjefond', 'skattefrittUttak', 'benyttetSkjermingsfradrag']]\
                .toPandas()

In [ ]:
#Kjører korrigering på numeriske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_number, missing_numbcount = missing_correction_number(skatt_dict['fondskonto'], exception_for=['skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel'])

In [ ]:
#Eksempel etter korrigering det skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel er unntatt korrigering
df_corrected_number\
                [['identifikator', 'skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel','skattepliktigTapVedRealisasjonAvOgUttakFraRentedel',\
                 'verdiFoerVerdsettingsrabattForAksjeOgAksjefond', 'skattefrittUttak', 'benyttetSkjermingsfradrag']]\
                .toPandas()

In [ ]:
%%time
#Eksempel som kjører korrigering av numeriske og bolske verdier for alle skatteobjekt og samler loggene i et eget datasett
corrected_missing_log = pd.DataFrame(columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])

for k in skatt_dict.keys():
    
    df, miss_bool = missing_correction_bool(skatt_dict[k])
    for l, m in miss_bool.items():
        pdrow = pd.DataFrame([[k, 'Boolske', l, m]], columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])
        corrected_missing_log = corrected_missing_log.append(pdrow)
        

    df, miss_number = missing_correction_number(df)
    for l, m in miss_number.items():
        pdrow = pd.DataFrame([[k, 'Number', l, m]], columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])
        corrected_missing_log.append(pdrow)
        
corrected_missing_log = corrected_missing_log.reset_index(drop=True)

In [ ]:
#Log av datesett
corrected_missing_log

In [ ]:
%run ../ssb_sparktools/editing/editing.py

In [ ]:
df_corrected_bool, missing_boolcount = spark_missing_correction_bool(skatt_dict['fondskonto']) 

In [ ]:
df_corrected_bool[['identifikator', 'overfoertFraBarn']].toPandas()

In [ ]:
#missing_correction_bool
#Kjører korrigering på bolske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_bool, missing_boolcount = missing_correction_bool(skatt_dict['fondskonto'])
for k, v in missing_boolcount.items():
    print(k, v)

In [ ]:
df_corrected_bool[['identifikator', 'overfoertFraBarn']].toPandas()

In [ ]:
#spark_missing_correction_bool
#Kjører korrigering på bolske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_bool, missing_boolcount = spark_missing_correction_bool(skatt_dict['fondskonto'])
for k, v in missing_boolcount.items():
    print(k, v)

In [ ]:
df_corrected_bool[['identifikator', 'overfoertFraBarn']].toPandas()

In [ ]:
#spark_missing_correction_number
#Kjører korrigering på numeriske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_number, missing_numbcount = spark_missing_correction_number(skatt_dict['fondskonto'])
for k, v in missing_numbcount.items():
    print(k, v)

In [ ]:
#Eksempel etter korrigering
df_corrected_number\
                [['identifikator', 'skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel','skattepliktigTapVedRealisasjonAvOgUttakFraRentedel',\
                 'verdiFoerVerdsettingsrabattForAksjeOgAksjefond', 'skattefrittUttak', 'benyttetSkjermingsfradrag']]\
                .toPandas()

In [ ]:
#missing_correction_number
#Kjører korrigering på numeriske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_number, missing_numbcount = missing_correction_number(skatt_dict['fondskonto'])
for k, v in missing_numbcount.items():
    print(k, v)

In [ ]:
#Eksempel etter korrigering
df_corrected_number\
                [['identifikator', 'skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel','skattepliktigTapVedRealisasjonAvOgUttakFraRentedel',\
                 'verdiFoerVerdsettingsrabattForAksjeOgAksjefond', 'skattefrittUttak', 'benyttetSkjermingsfradrag']]\
                .toPandas()

In [ ]:
%%time
#Eksempel som kjører korrigering av numeriske og bolske verdier for alle skatteobjekt og samler loggene i et eget datasett
corrected_missing_log = pd.DataFrame(columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])

for k in skatt_dict.keys():
    
    df, miss_bool = spark_missing_correction_bool(skatt_dict[k])
    for l, m in miss_bool.items():
        pdrow = pd.DataFrame([[k, 'Boolske', l, m]], columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])
        corrected_missing_log = corrected_missing_log.append(pdrow)
        

    df, miss_number = spark_missing_correction_number(df)
    for l, m in miss_number.items():
        pdrow = pd.DataFrame([[k, 'Number', l, m]], columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])
        corrected_missing_log.append(pdrow)
        
corrected_missing_log = corrected_missing_log.reset_index(drop=True)

In [ ]:
#Kjører på all data i prod
skattRaw = spark.read.path('/produkt/skatt/person/temp/tverr*').show(100, False)

In [ ]:
import pandas as pd
%run ../ssb_sparktools/editing/editing.py
%run ../ssb_sparktools/quality/quality.py
%run ../ssb_sparktools/processing/processing.py

In [ ]:
skattRaw = spark.read.path('/produkt/skatt/person/temp/tverrsnitt_skattemelding_All')

In [ ]:
skatt_dict = unpack_parquet(skattRaw, rootdf=True, rootvar=True)
spark

In [ ]:
%%time

#Eksempel som kjører korrigering av numeriske og bolske verdier for alle skatteobjekt og samler loggene i et eget datasett
corrected_missing_log = pd.DataFrame(columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])

for k in skatt_dict.keys():
    
    df, miss_bool = spark_missing_correction_bool(skatt_dict[k])
    for l, m in miss_bool.items():
        pdrow = pd.DataFrame([[k, 'Boolske', l, m]], columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])
        corrected_missing_log = corrected_missing_log.append(pdrow)
        

    df, miss_number = spark_missing_correction_number(df)
    for l, m in miss_number.items():
        pdrow = pd.DataFrame([[k, 'Number', l, m]], columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])
        corrected_missing_log.append(pdrow)
        
corrected_missing_log = corrected_missing_log.reset_index(drop=True)

In [ ]:
%%time
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

#Eksempel som kjører korrigering av numeriske og bolske verdier for alle skatteobjekt og samler loggene i et eget datasett

missing_corrections = []
for k in skatt_dict.keys():
    
    df, miss_bool = spark_missing_correction_bool(skatt_dict[k])
    for l, m in miss_bool.items():
        mRow = {}
        mRow['dataframe'] = k
        mRow['variabel'] = l
        mRow['korrigeringer'] = m
        missing_corrections.append(mRow)
        
    df, miss_number = spark_missing_correction_number(df)
    for l, m in miss_number.items():
        mRow = {}
        mRow['dataframe'] = k
        mRow['variabel'] = l
        mRow['korrigeringer'] = m
        missing_corrections.append(mRow)

#Oppretter dataframe for resultatet av gjennomgangen ovenfor

missing_variabler = [StructField('dataframe', StringType(), False),\
                   StructField('variabel', StringType(), True),\
                   StructField('korrigeringer', IntegerType(), False)]
missing_schema = StructType(missing_variabler)
rdd_missing = sc.parallelize(missing_corrections)

df_corrections = sqlContext.createDataFrame(rdd_missing, missing_schema)

In [ ]:
%%time
df_corrections.write\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path('/produkt/skatt/person/temp/logmissing/logall')

In [ ]:
df_corrections.show(20, False)

In [ ]:
corrected_missing_log

In [ ]:
%%time

#Eksempel som kjører korrigering av numeriske og bolske verdier for alle skatteobjekt og samler loggene i et eget datasett
corrected_missing_log = pd.DataFrame(columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])

for k in skatt_dict.keys():
    
    #df, miss_bool = missing_correction_bool(skatt_dict[k])
    #for l, m in miss_bool.items():
    #   pdrow = pd.DataFrame([[k, 'Boolske', l, m]], columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])
    #    corrected_missing_log = corrected_missing_log.append(pdrow)
        

    df, miss_number = missing_correction_number(skatt_dict[k])
    for l, m in miss_number.items():
        pdrow = pd.DataFrame([[k, 'Number', l, m]], columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])
        corrected_missing_log.append(pdrow)
        
corrected_missing_log = corrected_missing_log.reset_index(drop=True)

In [ ]:
numlist = ['personidentifikator']
for k in skatt_dict['aksjeIAksjonaerregisteret'].schema.fields:
    if str(k.dataType) in ['LongType', 'ByteType', 'ShortType', 'IntegerType', 'FloatType', 'DoubleType', 'DecimalType']:  
        numlist.append(k.name)
                    
df = skatt_dict['aksjeIAksjonaerregisteret'].select(numlist)
df_corrected = df.fillna(False)
df_felles = df.intersect(df_corrected)
df_corr = df.join(df_felles.select('personidentifikator'), 'personidentifikator', how='leftanti')
df_corr.toPandas()            

In [ ]:
%%time
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

#Eksempel som kjører korrigering av numeriske og bolske verdier for alle skatteobjekt og samler loggene i et eget datasett

missing_corrections = []
for k in skatt_dict.keys():
    
    df, miss_bool = spark_missing_correction_bool(skatt_dict[k])
    for l, m in miss_bool.items():
        mRow = {}
        mRow['dataframe'] = k
        mRow['variabel'] = l
        mRow['korrigeringer'] = m
        missing_corrections.append(mRow)
        
    df, miss_number = spark_missing_correction_number(df)
    for l, m in miss_number.items():
        mRow = {}
        mRow['dataframe'] = k
        mRow['variabel'] = l
        mRow['korrigeringer'] = m
        missing_corrections.append(mRow)

#Oppretter dataframe for resultatet av gjennomgangen ovenfor

missing_variabler = [StructField('dataframe', StringType(), False),\
                   StructField('variabel', StringType(), True),\
                   StructField('korrigeringer', IntegerType(), False)]
missing_schema = StructType(missing_variabler)
rdd_missing = sc.parallelize(missing_corrections)

df_corrections = sqlContext.createDataFrame(rdd_missing, missing_schema)

In [ ]:
df_felles.count()

In [ ]:
df_corr.count()

In [ ]:
from functools import reduce
df_agg_col = reduce(
    lambda a, b: a.union(b),
    (
        skatt_dict['aksjeIAksjonaerregisteret'].agg(
            F.count(F.when(F.isnull(c), c)).alias('ant_korrigeringer')
        ).select(F.lit(c).alias("variabel"), "ant_korrigeringer")
        for c in skatt_dict['aksjeIAksjonaerregisteret'].columns
    )
)

In [ ]:
df_agg_col.show(10, False)

In [ ]:
df_agg_col = df_agg_col.filter(F.col('ant_korrigeringer')>0)
df_agg_col.show(10, False)

In [ ]:
import pandas as pd
%run ../ssb_sparktools/editing/editing.py
%run ../ssb_sparktools/quality/quality.py
%run ../ssb_sparktools/processing/processing.py

In [ ]:
skattRaw = spark.read.path('/produkt/skatt/person/temp/tverrsnitt_skattemelding_All')

In [ ]:
skatt_dict = unpack_parquet(skattRaw, rootdf=True, rootvar=True)
spark

In [ ]:
df, miss_number = spark_missing_correction_number2(skatt_dict['aksjeIAksjonaerregisteret'])

In [ ]:
miss_number.show()

In [ ]:
df, miss_bool = spark_missing_correction_bool2(skatt_dict['aksjeIAksjonaerregisteret'])

In [ ]:
miss_bool.show()

In [ ]:
%%time
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

#Eksempel som kjører korrigering av numeriske og bolske verdier for alle skatteobjekt og samler loggene i et eget datasett

missing_corrections = []
for k in skatt_dict.keys():
    print(k)
    df, miss_bool = spark_missing_correction_bool2(skatt_dict[k])
    #for row in miss_bool.rdd.collect():
    #    mRow = {}
    #    mRow['dataframe'] = k
    #    mRow['variabel'] = row['variabel']
    #    mRow['korrigeringer'] = row['ant_korrigeringer']
    #    missing_corrections.append(mRow)
        
    df, miss_number = spark_missing_correction_number2(df)
    #for row in miss_number.rdd.collect():
    #    mRow = {}
    #    mRow['dataframe'] = k
    #    mRow['variabel'] = row['variabel']
    #    mRow['korrigeringer'] = row['ant_korrigeringer']
    #    missing_corrections.append(mRow)
    
    miss_all = miss_bool.union(miss_number)
    for row in miss_all.rdd.collect():
        mRow = {}
        mRow['dataframe'] = k
        mRow['variabel'] = row['variabel']
        mRow['korrigeringer'] = row['ant_korrigeringer']
        missing_corrections.append(mRow)
        
#Oppretter dataframe for resultatet av gjennomgangen ovenfor

missing_variabler = [StructField('dataframe', StringType(), False),\
                   StructField('variabel', StringType(), True),\
                   StructField('korrigeringer', IntegerType(), False)]
missing_schema = StructType(missing_variabler)
rdd_missing = sc.parallelize(missing_corrections)

df_corrections = sqlContext.createDataFrame(rdd_missing, missing_schema)

In [ ]:
%%time
misslogg_dict = {}
for k in skatt_dict.keys():
    df, miss_bool = spark_missing_correction_bool2(skatt_dict[k])    
    df, miss_number = spark_missing_correction_number2(df)
    miss_all = miss_bool.union(miss_number)
    misslogg_dict[k] = miss_all

In [ ]:
%%time
for k in misslogg_dict.keys():
    misslogg_dict[k].write\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path('/produkt/skatt/person/temp/logmissing/'+k)

In [ ]:
import pandas as pd
%run ../ssb_sparktools/editing/editing.py
%run ../ssb_sparktools/quality/quality.py
%run ../ssb_sparktools/processing/processing.py
spark

In [ ]:
skattRaw = spark.read.path('/produkt/skatt/person/temp/tverrsnitt_skattemelding_All')

In [ ]:
skatt_dict = unpack_parquet(skattRaw, rootdf=True, rootvar=True)

In [ ]:
%%time
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

missing_corrections = []
for k in skatt_dict.keys():
    
    skatt_dict[k], miss_bool = spark_missing_correction_bool(skatt_dict[k])
    for l, m in miss_bool.items():
        mRow = {}
        mRow['dataframe'] = k
        mRow['variabel'] = l
        mRow['korrigeringer'] = m
        missing_corrections.append(mRow)
        
    skatt_dict[k], miss_number = spark_missing_correction_number(skatt_dict[k])
    for l, m in miss_number.items():
        mRow = {}
        mRow['dataframe'] = k
        mRow['variabel'] = l
        mRow['korrigeringer'] = m
        missing_corrections.append(mRow)

#Oppretter dataframe for resultatet av gjennomgangen ovenfor

missing_variabler = [StructField('dataframe', StringType(), False),\
                   StructField('variabel', StringType(), True),\
                   StructField('korrigeringer', IntegerType(), False)]
missing_schema = StructType(missing_variabler)
rdd_missing = sc.parallelize(missing_corrections)

df_corrections = sqlContext.createDataFrame(rdd_missing, missing_schema)

In [ ]:
%%time
df_corrections.write\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path('/produkt/skatt/person/temp/logmissing/logall')

In [ ]:
df_corrections.show(10, False)

In [ ]:
test = spark.read.path('/produkt/skatt/person/temp/logmissing/logall')

In [ ]:
test.count()